## Combining NDVI and CDL data

In this lesson we'll combine our NDVI data with data from the [Cropland Data Layer](https://www.nass.usda.gov/Research_and_Science/Cropland/SARS1a.php) (CDL). The CDL is a land user/land cover raster at 30 meter resolution that classifies each pixel in a number of categories, including farmland for specific crops. The CDL puts out new CDL data each year; we'll be using data from the 2016 version.

To start, let's take the GeoJSON of the crop area you pasted into Lesson 2 and paste it here as well:

In [ ]:
import json
from shapely.geometry import shape

#crop_area_geojson = """
#<PASTE GEOJSON FROM LESSON 2 HERE>
#"""

crop_area_geojson = """
<YOUR TEXT HERE>
"""

crop_area_json = json.loads(crop_area_geojson)['features'][0]['geometry']
crop_area_ll = shape(crop_area_json)

The CDL data lives in [Amazon S3](https://aws.amazon.com/s3) and is publicly accessable. We can access through HTTP, which `rasterio` can handle directly. In addition, because the GeoTIFF is a [Cloud Optimized GeoTIFF](http://www.cogeo.org/) (COG), only the data we need will be pulled from S3. This makes remote access a lot faster.

First we'll need to figure out the CDL raster data's projection and reproject our area of interest into that projection. Then, we can use the `mask` method just like if the raster is on disk:

In [ ]:
import rasterio
from rasterio.mask import mask


cdl_url = 'https://s3.amazonaws.com/geotrellis-test/cdl-geotiff/CDLS_2016_30m.tif'

# Just read the CRS, so we can reproject our crop polygon to it.
with rasterio.open(cdl_url) as ds:
    cdl_crs = ds.crs

In [ ]:
import pyproj
from shapely.geometry import mapping
from musa import reproject_geom # From Lesson 1

cdl_proj = pyproj.Proj(cdl_crs)
lat_lng_proj = pyproj.Proj(init="epsg:4326")

crop_area = reproject_geom(crop_area_ll, lat_lng_proj, cdl_proj)

with rasterio.open(cdl_url) as ds:
    cdl_data, cdl_transform = mask(ds, [mapping(crop_area)], crop=True)

Let's render this piece of the CDL. We'll use a `ListedColormap` in matplot lib to use the legend colors that are specified 

In [ ]:
from matplotlib import colors

d = {1: "#ffd300",2: "#ff2626",3: "#00a8e5",4: "#ff9e0c",5: "#267000",6: "#ffff00",10: "#70a500",11: "#00af4c",12: "#dda50c",13: "#dda50c",14: "#7fd3ff",21: "#e2007c",22: "#896354",23: "#d8b56b",24: "#a57000",25: "#d69ebc",26: "#707000",27: "#ad007c",28: "#a05989",29: "#700049",30: "#d69ebc",31: "#d1ff00",32: "#7f99ff",33: "#d6d600",34: "#d1ff00",35: "#00af4c",36: "#ffa5e2",37: "#a5f28c",38: "#00af4c",39: "#d69ebc",41: "#a800e5",42: "#a50000",43: "#702600",44: "#00af4c",45: "#b27fff",46: "#702600",47: "#ff6666",48: "#ff6666",49: "#ffcc66",50: "#ff6666",51: "#00af4c",52: "#00ddaf",53: "#54ff00",54: "#f2a377",55: "#ff6666",56: "#00af4c",57: "#7fd3ff",58: "#e8bfff",59: "#afffdd",60: "#00af4c",61: "#bfbf77",63: "#93cc93",64: "#c6d69e",65: "#ccbfa3",66: "#ff00ff",67: "#ff8eaa",68: "#ba004f",69: "#704489",70: "#007777",71: "#b29b70",72: "#ffff7f",74: "#b5705b",75: "#00a582",76: "#ead6af",77: "#b29b70",81: "#f2f2f2",82: "#9b9b9b",83: "#4c70a3",87: "#7fb2b2",88: "#e8ffbf",92: "#00ffff",111: "#4c70a3",112: "#d3e2f9",121: "#9b9b9b",122: "#9b9b9b",123: "#9b9b9b",124: "#9b9b9b",131: "#ccbfa3",141: "#93cc93",142: "#93cc93",143: "#93cc93",152: "#c6d69e",176: "#e8ffbf",190: "#7fb2b2",195: "#7fb2b2",204: "#00ff8c",205: "#d69ebc",206: "#ff6666",207: "#ff6666",208: "#ff6666",209: "#ff6666",210: "#ff8eaa",211: "#334933",212: "#e57026",213: "#ff6666",214: "#ff6666",216: "#ff6666",217: "#b29b70",218: "#ff8eaa",219: "#ff6666",220: "#ff8eaa",221: "#ff6666",222: "#ff6666",223: "#ff8eaa",224: "#00af4c",225: "#ffd300",226: "#ffd300",227: "#ff6666",229: "#ff6666",230: "#896354",231: "#ff6666",232: "#ff2626",233: "#e2007c",234: "#ff9e0c",235: "#ff9e0c",236: "#a57000",237: "#ffd300",238: "#a57000",239: "#267000",240: "#267000",241: "#ffd300",242: "#000099",243: "#ff6666",244: "#ff6666",245: "#ff6666",246: "#ff6666",247: "#ff6666",248: "#ff6666",249: "#ff6666",250: "#ff6666",251: "#ffd300",252: "#267000",253: "#a57000",254: "#267000ff"}
l = list(d.items())
cmap = colors.ListedColormap(list(map(lambda x: x[1], l)))
boundaries = [0] + list(map(lambda x: x[0], l))
norm = colors.BoundaryNorm(boundaries, cmap.N)


In [ ]:
from musa import show_image
show_image(cdl_data[0], cmap=cmap)

We can take a look at the histogram of values to get a sense of the data distribution:

In [ ]:
from musa import show_histogram # From Lesson 2
import numpy as np

#show_histogram(np.ravel(cdl_data[~(cdl_data == 0)]), no_data_value=0)
show_histogram(cdl_data, no_data_value=0)

This may not be very telling, as it doesn't show the names of the classes. We can compute the counts of each class by using the `np.unique` method

In [ ]:
from musa import cdl_values_to_crops

flattened = np.ravel(cdl_data[~(cdl_data == 0)])
values_arr, values_counts = np.unique(flattened, return_counts=True)

# Zip the values together into a list of tuples
values_to_counts = zip(values_arr, values_counts)

# Use a list comprehension to transform crop IDs in to crop names
crop_counts = [(cdl_values_to_crops[crop_id], count) for (crop_id, count) in values_to_counts]
crop_counts

Notice the use of [List Comprehensions](https://docs.python.org/3.6/tutorial/datastructures.html#list-comprehensionsa) above, which is a shorthand way to map over values in a collection.

For further processing, we can use the [pandas](https://pandas.pydata.org/) library, a popular library in the Data Scientist's toolkit. It features a `DataFrame` type that allows you to work with data in a tabular fashion, much like a database table or a spreadsheet.

Here we create a `DataFrame` from our crop count data:

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(crop_counts, columns=['crop','count'])
df

Similar to histograms numpy arrays, we can use matplotlib to plot a bar graph of our crop counts:

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
df.plot.bar(x='crop', y='count', legend=False)
plt.show()

That bar graph likely contains a lot of columns and is unreadable. To get around that, we can filter the `DataFrame` by a threshold to only look at classes that have a number of cells representing them:

In [ ]:
THRESHOLD = 10000
filtered = df[df['count'] > THRESHOLD]

plt.figure()
filtered.plot.bar(x='crop', y='count', legend=False)
plt.show()

You may need to play around with the `THRESHOLD` value to get a good set of classes.

From that bar graph, pull out some crop names that we'll use for futher analysis, and replace the list below with your chosen crop names:

In [ ]:
target_crops = ['Cotton',
                'Corn',
                'Safflower',
                'Tomatoes',
                'Almonds',
                'Pistachios']

## Compute NDVI statistics per crop

We'll now combine NDVI data and CDL data to inspect NDVI per crop type.

Load up the landsat scenes, cropped to the area of interest, and compute the NDVI:

In [ ]:
import os

landsat_scene_name = "LC08_L1TP_042035_20171022_20171107_01_T1"

red_path = os.path.join("/home/hadoop/data/", 
                        "{}_B4.TIF".format(landsat_scene_name))
ir_path = os.path.join("/home/hadoop/data/", 
                       "{}_B5.TIF".format(landsat_scene_name))

In [ ]:
with rasterio.open(red_path) as ds:
    landsat_crs = ds.crs
    
landsat_proj = pyproj.Proj(landsat_crs)

landsat_crop_area = reproject_geom(crop_area_ll, lat_lng_proj, landsat_proj)

with rasterio.open(red_path) as ds:
    (red_data, landsat_transform) = mask(ds, [mapping(landsat_crop_area)], crop=True)
    
with rasterio.open(ir_path) as ds:
    (ir_data, _) = mask(ds, [mapping(landsat_crop_area)], crop=True)


In [ ]:
from musa import compute_ndvi
ndvi = compute_ndvi(red_data[0], ir_data[0])

## Aligning the NDVI and CDL rasters

In order to process rasters together, they need to be pixel aligned. Even though both CDL and Landsat data is at 30 meters, and is cropped to the same area, the pixel grids most likely do not align. In fact, the shape of the data most likely is different:

In [ ]:
print("{} == {} ? {}".format(ndvi.shape, cdl_data[0].shape, ndvi.shape == cdl_data[0].shape))

The projects are also different. So we'll need to reproject one of the rasters into the other raster's space, and resample to align the pixel grids. 

Let's reproject the NDVI data to match the CDL data with the `reproject` method from `rasterio`:

In [ ]:
from rasterio.warp import reproject
from rasterio.enums import Resampling

In [ ]:
warped_ndvi = np.zeros(cdl_data[0].shape)
reproject(source=ndvi,
          src_transform=landsat_transform,
          src_crs=landsat_crs,
          destination=warped_ndvi,
          dst_transform=cdl_transform,
          dst_crs=cdl_crs,
          resampling=Resampling.bilinear)

## Including NDVI statistics in the DataFrame

The code below calculates the min, max and mean NDVI values for each crop, and adds them to the `pandas` `DataFrame` that contains our crop counts:

In [ ]:
from musa import crops_to_cdl_values

def ndvi_stats(crop):
    v = crops_to_cdl_values[crop]
    masked_ndvi = np.ravel(warped_ndvi[(cdl_data[0] == v) & (~np.isnan(warped_ndvi))])
    return pd.Series({'min': np.min(masked_ndvi),
                      'max': np.max(masked_ndvi),
                      'mean': np.mean(masked_ndvi)})

In [ ]:
pd.concat([df, df['crop'].apply(ndvi_stats)], axis=1)

More revealing than a table is what happens when we plot NDVI histograms per crop:

In [ ]:
min_ndvi = np.min(warped_ndvi[~np.isnan(warped_ndvi)])
max_ndvi = np.max(warped_ndvi[~np.isnan(warped_ndvi)])
bins = np.linspace(min_ndvi, max_ndvi, 100)

#target_crops = ['Corn']
f, axarr = plt.subplots(len(target_crops), 1, figsize=(15,15), sharex=True)

ndvis = []
for crop in target_crops:
    v = crops_to_cdl_values[crop]
    ndvis.append(np.ravel(warped_ndvi[(cdl_data[0] == v) & (~np.isnan(warped_ndvi))]))
    

for i, crop in enumerate(target_crops):
    v = crops_to_cdl_values[crop]
    target_values = warped_ndvi[(cdl_data[0] == v) & (~np.isnan(warped_ndvi))]
    axarr[i].hist(target_values, bins, label=crop)
    axarr[i].set_title(crop)

plt.show()

The data you likely see here indicates that some crop fields have a even spread of NDVI values, or have one grouping at a low NDVI value. However some crop fields have two groupings of NDVI values: places where the crop has been harvested, and places where the crop has grown and yet to be harvested*.

*I'm not an agronomyst, so take my reading of the data with a large grain of salt.

## Finding Harvested fields image segmentation

Pick one crop that exhibits this behavior - two distinct "peaks" in the histogram. Replace the crop name below:

In [ ]:
crop_name = "Cotton"

We can use [K-Means clustering](https://en.wikipedia.org/wiki/K-means_clustering) to find the two cluster centers in our data. Since we only have 1 dimensional data, K-means clustering represents a [Jenks natural breaks](https://en.wikipedia.org/wiki/Jenks_natural_breaks_optimization) seperation of the data.

[scikit-learn](http://scikit-learn.org/stable/index.html) has a KMeans class that will let us perform k-means on our data easily:

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# We only want pixels of our crop, that are valid numbers.
mask_cond = (~np.isnan(warped_ndvi)) &(cdl_data[0] == crops_to_cdl_values[crop_name])

# Flatten out the data values, and reshape as needed for KMeans
crop_ndvi = np.ravel(warped_ndvi[mask_cond]).reshape(-1, 1)

# Run the unsupervised learning
kmeans = KMeans(n_clusters=2, random_state=0).fit(crop_ndvi)

Now that we have 2 cluster centers, we can find the value between them that represents the boundary point of the lower and higher groups.

In [ ]:
break_point = np.sum(kmeans.cluster_centers_) / 2
break_point

In this analysis, we are considering pixels containing our target crop classification with NDVI values less than or equal to the break_point to be "harvested field", and the similarly classified pixels with NDVI greater than the break point to be "unharvested field". 

Let's encode that information into two masks:

In [ ]:
from scipy import ndimage as ndi

def create_mask(condition):
    unharvested_mask = np.zeros(warped_ndvi.shape, dtype='uint8')
    unharvested_mask[mask_cond & condition] = 1
    # Fill in holes in the data to try and create continuous regions.
    return ndi.binary_fill_holes(unharvested_mask).astype('uint8')

unharvested_mask = create_mask(warped_ndvi > break_point)
harvested_mask = create_mask(warped_ndvi <= break_point)

We should see the harvested and unharvested fields that have been segmented out:

In [ ]:
show_image(ndi.binary_fill_holes(unharvested_mask))
show_image(ndi.binary_fill_holes(harvested_mask))

[skimage](http://scikit-image.org/) is an image processing library that includes methods like creating contour lines. We use that functionality here to plot the contours of the fields only our images:

In [ ]:
from skimage.measure import find_contours

def plot_contours(img_mask, title):
    fig, ax = plt.subplots(figsize=(16, 16))
    ax.set_title(title)
    ax.imshow(warped_ndvi, interpolation='none', cmap=cmap, clim=(-1.0, 1.0))

    ax.axis('tight')

    for n, contour in enumerate(find_contours(img_mask, 0)):
        ax.plot(contour[:, 1], contour[:, 0], linewidth=4)

plot_contours(unharvested_mask, "Unharvested {} Fields".format(crop_name))
plot_contours(harvested_mask, "Harvested {} Fields".format(crop_name))
plt.show()


We can go head and write out our data as a shapefile. For this we'll use a library called [fiona](https://github.com/Toblerity/Fiona), a library for reading and writing vector data.

In [ ]:
import fiona
import rasterio.features

schema = {"geometry": "Polygon", "properties": {"value": "int"}}

# Create a raster with values:
#  0 = Not containing target crop
#  1 = Contains unharvested crop
#  2 = Contains harvested crop

mask_img = unharvested_mask.copy()
mask_img[harvested_mask == 1] = 2

features = []
for (geom, value) in rasterio.features.shapes(mask_img, transform=cdl_transform):
    if value != 0:
        ll_geom = reproject_geom(shape(geom), cdl_proj, lat_lng_proj)
        features.append({ 'geometry': mapping(ll_geom), 
                          'properties': { 'value': value} })

# Write as a shapefile
with fiona.open("/home/hadoop/data/fields.shp", "w", "ESRI Shapefile",
                crs=cdl_crs.data, schema=schema) as out_file:
    out_file.writerecords(features)

We can also write to GeoJSON using `json.dumps`. Here we set a property in the GeoJSON that will assign colors to the field boundaries in geojson.io. Go ahead and save off the GeoJSON, load the file up in geojson.io, and see if the fields match up with the satellite imagery basemap.

In [ ]:
# Write as a GeoJSON FeatureCollection
def set_feature_type(f):
    f['type'] = "Feature"
    if f['properties']['value'] == 1:
        f['properties']['fill'] = "#00FF00"
    else:
        f['properties']['fill'] = "#FF0000"
    return f

feature_collection = {"type": "FeatureCollection", 
                      "features": list(map(set_feature_type, features)) }
with open("/home/hadoop/data/fields.geojson", 'w') as f:
    f.write(json.dumps(feature_collection, indent=4))